In [26]:
import requests
import json
import pandas as pd

In [20]:
url = 'http://localhost:4000/graphql'

In [28]:
moduleData = requests.post(url, json={'query': '{ module(input:{}){id, name, hours, objectives, description} }'}).json()

print(json.dumps(moduleData.get('data').get('module'), indent=2))

[
  {
    "id": "644827afc3faa4c9ea96a800",
    "name": "Linear Regression",
    "hours": 0.75,
    "objectives": [
      "regression",
      "algebra",
      "statistics"
    ],
    "description": "This module is designed to introduce students to linear regression, a powerful statistical technique used to model the relationship between two variables. Through interactive lessons and hands-on activities, students will learn how to build linear regression models, interpret their results, and use them to make predictions and draw conclusions about the data. The module will cover the assumptions and limitations of linear regression, as well as techniques for validating and improving the model. Students will also learn about different types of linear regression, including simple and multiple regression, and how to apply them to real-world problems. By the end of the module, students will have a strong understanding of linear regression and be able to apply it to a wide range of fields, incl

In [31]:
moduleDF = pd.DataFrame(moduleData.get('data').get('module'))

moduleDF.head()

,id,name,hours,objectives,description
0,644827afc3faa4c9ea96a800,Linear Regression,0.75,"[regression, algebra, statistics]",This module is designed to introduce students ...
1,64496079d5fb014e40d5ea39,Statistical Analysis,1.50,[statistics],This module is designed to teach students the ...
2,644960add5fb014e40d5ea3a,Logarithms,2.00,"[algebra, logarithms]",This module is designed to introduce students ...
3,644960bcd5fb014e40d5ea3b,Arithmatics,0.50,[algebra],This module is designed to teach students the ...
4,64496111d5fb014e40d5ea3c,Euclidean distance,2.25,"[linear algebra, statistics, regression]","In this module, we'll be learning about the Eu..."


In [23]:
sectionData = requests.post(url, json={'query': '{ section(input:{}){id, sectionName, description, intro, keywords, objectives} }'}).json()

print(json.dumps(sectionData.get('data').get('section'), indent=2))

[
  {
    "id": "64480fd85ee3f93a77784aeb",
    "sectionName": "Course Introduction and Design",
    "description": "Assess the effectiveness of your past, current, and future projects based on universal success factors and global contingency considerations,  Intelligently evaluate projects for inclusion, and continued investment, in an organization\u2019s portfolio,  Revise project plans to accommodate time and resource conflicts, Evaluate project performance based on a balanced set of key performance indicators, Apply critical thinking to assess project situations and devise appropriate tactical and strategic responses, Present yourself to a potential employer as someone who is capable of leading projects",
    "intro": "",
    "keywords": [
      "expectations",
      "management",
      "conceptual"
    ],
    "objectives": [
      "Discuss high level and integration of topics in course design",
      "Understand conceptual approach to treatment of project management",
      "Recog

In [32]:
sectionDF = pd.DataFrame(sectionData.get('data').get('section'))

sectionDF.head()

,id,sectionName,description,intro,keywords,objectives
0,64480fd85ee3f93a77784aeb,Course Introduction and Design,"Assess the effectiveness of your past, current...",,"[expectations, management, conceptual]",[Discuss high level and integration of topics ...
1,64492dab53da498461a3e3c1,Test 2,Some wise words,Wise words,[],[]
2,645116ec2a4785d4b7799759,Section 1,"In this module, we introduce to the area of Op...",,[],[]
3,645116f22a4785d4b779975a,Section 2,"In this module, we introduce the area of Linea...",,[],[]
4,645116f62a4785d4b779975b,Section 3,,,[],[]


In [49]:
objectivesList = list()

for obj in moduleDF.objectives:
    for elm in obj:
        objectivesList.append(elm)

objectivesDF = pd.DataFrame(objectivesList)

objectivesDF.head(10)

,0
0,regression
1,algebra
2,statistics
3,statistics
4,algebra
5,logarithms
6,algebra
7,linear algebra
8,statistics
9,regression
